In [ ]:
from pathlib import Path
import json
import sys

import gtfstk as gt
import pandas as pd
import numpy as np
import shapely.geometry as sg
import googlemaps
import polyline
import requests
import folium

sys.path.append('../')

import gtfs_map_matcher as mm

%load_ext autoreload
%autoreload 2

DATA_DIR = Path('../data')
OUT_DIR = Path('../output')
SECRETS_PATH = Path('../secrets.json')

In [2]:
path = DATA_DIR/'wellington_gtfs_20171016.zip'
feed = gt.read_gtfs(path, dist_units='km')
feed.assess_quality()

In [3]:
# shapes_g = feed.shapes_to_geojson()
# path = Path('../wellington_shapes_20171016.geojson')
# with path.open('w') as tgt:
#     json.dump(shapes_g, tgt)

In [4]:
# t = feed.trips.merge(feed.routes)
# t = t[t['route_type'] == 3].copy()
# print(t.shape[0])
# t = t.iloc[:100]
# st = mm.build_sample_points(feed, t.trip_id, point_dist=0.25)
# st

In [5]:
# Visually test ``build_sample_points()``

t = feed.trips.merge(feed.routes)
t = t[t['route_type'] == 3].sample(frac=0.01)
%time sample_points_1 = mm.build_sample_points(feed, t.trip_id, 4)
%time sample_points_2 = mm.build_sample_points(feed, t.trip_id, 100)
%time sample_points_3 = mm.build_sample_points(feed, t.trip_id, point_dist=0.1)


CPU times: user 1.48 s, sys: 12 ms, total: 1.5 s
Wall time: 1.48 s
CPU times: user 3.54 s, sys: 16 ms, total: 3.56 s
Wall time: 3.54 s
CPU times: user 4.84 s, sys: 8 ms, total: 4.85 s
Wall time: 4.83 s


In [7]:
sp = list(sample_points_1.keys())[0]
l1 = sg.LineString(sample_points_1[sp])
l2 = sg.LineString(sample_points_2[sp])
l3 = sg.LineString(sample_points_3[sp])

# Get actual trip shape
p = mm.get_stop_patterns(feed)
shid = p.loc[p['stop_pattern'] == sp, 'shape_id'].unique()[0]
la = feed.build_geometry_by_shape(shape_ids=[shid], use_utm=False)[shid]

mappy = folium.Map(location=sample_points_1[sp][0][::-1], zoom_start=14)
folium.GeoJson(sg.mapping(l1), 
  style_function=lambda x: {'color': 'red'}).add_to(mappy)
folium.GeoJson(sg.mapping(l2), 
  style_function=lambda x: {'color': 'orange'}).add_to(mappy)
folium.GeoJson(sg.mapping(l3), 
  style_function=lambda x: {'color': 'yellow'}).add_to(mappy)
folium.GeoJson(sg.mapping(la), 
  style_function=lambda x: {'color': 'green'}).add_to(mappy)
mappy

In [8]:
# Visually test Mapzen

lm = sg.LineString(mm.match_with_mapzen(sample_points_2[sp], 
  mm.get_secret(SECRETS_PATH, 'MAPZEN_API_KEY')))

mappy = folium.Map(location=sample_points_1[sp][0][::-1], zoom_start=14)
folium.GeoJson(sg.mapping(la), 
  style_function=lambda x: {'color': 'green'}).add_to(mappy)
folium.GeoJson(sg.mapping(lm), 
  style_function=lambda x: {'color': 'red'}).add_to(mappy)
mappy


In [9]:
# Visually test OSRM

lm = sg.LineString(mm.match_with_osrm(sample_points_2[sp]))

mappy = folium.Map(location=sample_points_1[sp][0][::-1], zoom_start=14)
folium.GeoJson(sg.mapping(la), 
  style_function=lambda x: {'color': 'green'}).add_to(mappy)
folium.GeoJson(sg.mapping(lm), 
  style_function=lambda x: {'color': 'red'}).add_to(mappy)
mappy


In [10]:
# Visually test Mapbox

lm = sg.LineString(mm.match_with_mapbox(sample_points_2[sp], 
  mm.get_secret(SECRETS_PATH, 'MAPBOX_API_KEY')))

mappy = folium.Map(location=sample_points_1[sp][0][::-1], zoom_start=14)
folium.GeoJson(sg.mapping(la), 
  style_function=lambda x: {'color': 'green'}).add_to(mappy)
folium.GeoJson(sg.mapping(lm), 
  style_function=lambda x: {'color': 'red'}).add_to(mappy)
mappy


In [19]:
# Visually test Google

lm = sg.LineString(mm.match_with_google(sample_points_2[sp], 
  mm.get_secret(SECRETS_PATH, 'GOOGLE_API_KEY')))

mappy = folium.Map(location=sample_points_1[sp][0][::-1], zoom_start=14)
folium.GeoJson(sg.mapping(la), 
  style_function=lambda x: {'color': 'green'}).add_to(mappy)
folium.GeoJson(sg.mapping(lm), 
  style_function=lambda x: {'color': 'red'}).add_to(mappy)
mappy
